In [1]:
import pandas as pd
import datetime
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff


In [2]:
X=pd.read_csv('X.csv')

a1=list(X.Region.unique())
regions_dict=dict(zip(range(1,len(a1)+1),a1))
def my_date(dt):
    a= dt.split('-')
    return datetime.date(int(a[0]),int(a[1]),int(a[2]))
X.Date=X.Date.apply(my_date)
myday = list(X.tail(1).Date)[0]
print(regions_dict)

{1: 'Адыгея', 2: 'Алтай', 3: 'Алтайский край', 4: 'Амурская обл.', 5: 'Архангельская обл.', 6: 'Астраханская обл.', 7: 'Башкортостан', 8: 'Белгородская обл.', 9: 'Брянская обл.', 10: 'Бурятия', 11: 'Владимирская обл.', 12: 'Волгоградская обл.', 13: 'Вологодская обл.', 14: 'Воронежская обл.', 15: 'Дагестан', 16: 'Еврейская АО', 17: 'Забайкальский край', 18: 'Ивановская обл.', 19: 'Ингушетия', 20: 'Иркутская обл.', 21: 'Кабардино-Балкария', 22: 'Калининградская обл.', 23: 'Калмыкия', 24: 'Калужская обл.', 25: 'Камчатский край', 26: 'Карачаево-Черкессия', 27: 'Карелия', 28: 'Кемеровская обл.', 29: 'Кировская обл.', 30: 'Коми', 31: 'Костромская обл.', 32: 'Краснодарский край', 33: 'Красноярский край', 34: 'Крым', 35: 'Курганская обл.', 36: 'Курская обл.', 37: 'Ленинградская обл.', 38: 'Липецкая обл.', 39: 'Магаданская обл.', 40: 'Марий Эл', 41: 'Мордовия', 42: 'Москва', 43: 'Московская обл.', 44: 'Мурманская обл.', 45: 'Ненецкий АО', 46: 'Нижегородская обл.', 47: 'Новгородская обл.', 48: '

In [3]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14051 entries, 0 to 14050
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Date                14051 non-null  object 
 1   Confirmed           14051 non-null  int64  
 2   Recovered           14051 non-null  int64  
 3   Deaths              14051 non-null  int64  
 4   Day_deaths          14051 non-null  int64  
 5   Day_confirmed       14051 non-null  int64  
 6   Day_recovered       14051 non-null  int64  
 7   Rt                  12894 non-null  float64
 8   Rt2                 13004 non-null  float64
 9   Rate_conf           14051 non-null  float64
 10  Rate_ill            14051 non-null  float64
 11  Region              14051 non-null  object 
 12  Remaining_ill       14051 non-null  int64  
 13  MA7_dayconfirmed    13535 non-null  float64
 14  MA14_dayconfirmed   12933 non-null  float64
 15  MA7_remaining_ill   13535 non-null  float64
 16  MA14

In [4]:
covid_rates = pd.DataFrame()
mask = (X.Date==myday)&(X.Region!='Россия')
today_cases = X[mask]
#covid_rates.reset_index(drop=True)

covid_rates['По количеству текущих <br>больных'] = today_cases.sort_values(by=['Remaining_ill'],ascending = False).Region.reset_index(drop=True)
covid_rates['По количеству новых'] = today_cases.sort_values(by=['Day_confirmed'],ascending = False).Region.reset_index(drop=True)
covid_rates['По умершим за день'] = today_cases.sort_values(by=['Day_deaths'],ascending = False).Region.reset_index(drop=True)
covid_rates['По выздоровевшим<br>за день'] = today_cases.sort_values(by=['Day_recovered'],ascending = False).Region.reset_index(drop=True)
param = {'xgap':0}
fig =  ff.create_table(covid_rates.head(15),height_constant=200,**param)
fig.update_layout(
    
                  title_text ='Таблица рейтингов заболеваемости COVID-19 по регионам',
                  margin = {'t':50, 'b':100},
                  title_x = 0.5,
                  title_y= 0.95,
                  title_xanchor = "center",
                  title_yanchor = "top", 
                  
                  width = 990, height = 600,template = 'gridon',
    
    
                  xaxis_title='',yaxis_title = ''
                  )
for i in range(0,4):
    fig.layout.annotations[i].font.size = 10
fig.show()




In [5]:
region_number = 10
regions_towatch = list(covid_rates['По количеству текущих <br>больных'].head(region_number))
regions_towatch.append("Россия")
regions_towatch

['Москва',
 'Московская обл.',
 'Санкт-Петербург',
 'Свердловская обл.',
 'Челябинская обл.',
 'Саратовская обл.',
 'Иркутская обл.',
 'Волгоградская обл.',
 'Ставропольский край',
 'Ульяновская обл.',
 'Россия']

In [6]:
mask = (X.Date>datetime.date(2020,5,1))&(X.Region.isin(regions_towatch))
#mask = (X.Region.isin(regions_towatch))

fig = px.line(X[mask], y="Day_confirmed", x="Date",
                             color="Region", line_group="Region",
              line_shape="spline", render_mode="notebook",labels={'Region':'Регион:'},
              color_discrete_map={
                "Москва": "red",
                "Московская обл.": "magenta",
                "Санкт-Петербург": "goldenrod",
                "Нижегородская обл.": "darkblue",
                "Свердловская обл.": "green",
                 'Россия': 'aquamarine'},
             title="График количества новых больных по 5 регионам с наибольшим количеством заболеваний")

fig.update_layout(
    annotations=[
            dict(
            x=0.5,
            y=-0.15,
            showarrow=False,
            text="Данные: Яндекс",
            xref="paper",
            yref="paper")],
           
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                  
                  width = 990, height = 600,template = 'gridon',
    
    
                  xaxis_title='',yaxis_title = ''
)

fig.show()






In [7]:
#mask = (X.Date>datetime.date(2020,5,1))&(X.Region.isin(regions_towatch))

colors_dict = {"Москва": "red",
                "Московская обл.": "magenta",
                "Санкт-Петербург": "goldenrod",
                "Нижегородская обл.": "darkblue",
                "Свердловская обл.": "green",
                 'Россия':'aquamarine'}
fig = go.Figure()
for i in regions_towatch:
    mask = X['Region'] == i
        
    fig.add_trace(
        
        go.Bar(
        name = f'{i}',
        x=X[mask].Date,
        y=X[mask].Rt,
        text=round(X[mask].Rt,3),
        textposition='inside',
        marker_color= colors_dict.get(i),
        
        showlegend=True))




                  
fig.update_layout(
    annotations=[
        dict(
            x='2020-04-13',
            y=1.717,
            xref="x",
            yref="y",
            text="Введение<br>пропускной<br>системы в Москве",
            showarrow=True,
            arrowhead=2,
            ax=0,
            ay=-60),
        dict(
            x='2020-06-09',
            y=0.942,
            xref="x",
            yref="y",
            text="Отмена<br>пропускной<br>системы в Москве",
            showarrow=True,
            arrowhead=2,
            ax=0,
            ay=-60),
            dict(x=0.5,y=-0.15,
            showarrow=False,
            text="Данные : Яндекс <br> формула расчета из методических рекомендациях Роспотребнадзора.",
            xref="paper",
            yref="paper")],
                  title = "Коэффициент распространения (Rt) по 5 регионам ,<br> с максимальным количеством новых заболеваний ",
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                 # legend_x = 0.05,legend_y = 0.9,
                  width = 990, height = 600,template = 'gridon',
    
    
                 # xaxis_title='Коэффициент рассчитан, исходя из данных оперативного штаба и <br>' + 
                               #' формулы расчета в методических рекомендациях Роспотребнадзора. ',
    yaxis_title = ' '
)


fig.show()


In [8]:
print('Все регионы по которым есть статистика:')
print('*'*40)
print(regions_dict.items())
print('*'*40)
print('Рассматриваем регионы: ')
for i in regions_dict.keys():
    if regions_dict.get(i) in regions_towatch:
        print(i,regions_dict.get(i))


my_region = regions_dict[41]
print('Выбрали регион: ',my_region)


Все регионы по которым есть статистика:
****************************************
dict_items([(1, 'Адыгея'), (2, 'Алтай'), (3, 'Алтайский край'), (4, 'Амурская обл.'), (5, 'Архангельская обл.'), (6, 'Астраханская обл.'), (7, 'Башкортостан'), (8, 'Белгородская обл.'), (9, 'Брянская обл.'), (10, 'Бурятия'), (11, 'Владимирская обл.'), (12, 'Волгоградская обл.'), (13, 'Вологодская обл.'), (14, 'Воронежская обл.'), (15, 'Дагестан'), (16, 'Еврейская АО'), (17, 'Забайкальский край'), (18, 'Ивановская обл.'), (19, 'Ингушетия'), (20, 'Иркутская обл.'), (21, 'Кабардино-Балкария'), (22, 'Калининградская обл.'), (23, 'Калмыкия'), (24, 'Калужская обл.'), (25, 'Камчатский край'), (26, 'Карачаево-Черкессия'), (27, 'Карелия'), (28, 'Кемеровская обл.'), (29, 'Кировская обл.'), (30, 'Коми'), (31, 'Костромская обл.'), (32, 'Краснодарский край'), (33, 'Красноярский край'), (34, 'Крым'), (35, 'Курганская обл.'), (36, 'Курская обл.'), (37, 'Ленинградская обл.'), (38, 'Липецкая обл.'), (39, 'Магаданская обл.'

In [9]:
mask = X.Region ==my_region
#mask = (X.Date>datetime.date(2020,5,1)&(X.Region.isin(regions_towatch))
#mask = (X.Date>datetime.date(2020,5,1)&(X.Region==my_region)

fig = px.bar(X[mask], x='Date', 
             y="Day_confirmed",color ='Day_confirmed',
             title=f'{my_region}: '+'график количества новых заболевших COVID-19 по датам',
              color_continuous_scale= 'jet',text = 'Day_confirmed',
             labels = {'Day_confirmed' : "Новые заболевшие",'Remaining_ill':'Текущие больные'})
fig.add_trace(
    go.Scatter(
        name='14-дневная скользящая средняя',
        x=X[mask].Date,
        y=X[mask]['MA14_dayconfirmed'],
        mode="lines",
        
        line=go.scatter.Line(color="green"),
        showlegend=True)
)
fig.add_trace(
    go.Scatter(
        name = '7-дневная скользящая средняя',
        x=X[mask]['Date'],
        y=X[mask]['MA7_dayconfirmed'],
        mode="lines",
        
        line=go.scatter.Line(color="red"),
        showlegend=True)
)
fig.update_layout(
    annotations=[
#         dict(
#             x='2020-04-13',
#             y=1355,
#             xref="x",
#             yref="y",
#             text="Введение<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=0,
#             ay=-60),
#         dict(
#             x='2020-06-09',
#             y=1572,
#             xref="x",
#             yref="y",
#             text="Отмена<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=0,
#             ay=-90),
        dict(
            x=0.5,
            y=-0.15,
            showarrow=False,
            text="Данные: Яндекс",
            xref="paper",
            yref="paper"),
        
                ],
    
    
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                  legend_x = 0.15,legend_y = 0.95,
                  width = 990, height = 600,template = 'gridon',
    
    
                  xaxis_title='',yaxis_title = ' '
)


fig.show()




In [10]:
mask = X.Region ==my_region
#mask = (X.Date>datetime.date(2020,5,1))&(X.Region==my_region)
#Y = X[mask]

#Y['base'] = Y['Change_dayconf']*Y['Change_dayconf'].apply(lambda d:0 if d>0 else 1)
#Y['Date']= pd.to_datetime(Y.Date)
fig = go.Figure()
fig.add_trace(
              go.Bar(x = X[mask].Date,y=X[mask].Change_dayconf,text = X[mask]['Change_dayconf'],textposition = 'inside',hoverinfo='x+y',
                
                
                 marker_color = X[mask]['Change_dayconf'],marker_colorscale = 'Temps',marker_colorbar ={'tickmode':'auto',
                                    'title':{'text':'Изменение','side':'top'}}
                    )   
                )
fig.update_layout(
   annotations=[
        
        dict(x=0.5,y=-0.15,
            showarrow=False,
            text="",
            xref="paper",
            yref="paper")],
                  title=f'{my_region}: динамика новых заболевших COVID-19 по дням<br>(изменение относительно предыдущего дня)',
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                  legend_x = 0.05,legend_y = 0.98,
                  width = 990, height = 600,template = 'gridon',
    
    
                  xaxis_title='Данные: Яндекс',
    yaxis_title = ' '
)

fig.show()






In [11]:
mask = X.Region ==my_region
#mask = (X.Date>'2020-05-01')&(X.Region.isin(regions_towatch))

fig = px.bar(X[mask], x='Date', y="Remaining_ill",color ='Remaining_ill',
             title=f'{my_region}: '+'график количества текущих больных COVID-19 по датам',
              color_continuous_scale= 'tempo',text = 'Remaining_ill',
            labels = {'Day_confirmed' : "Новые заболевшие",'Remaining_ill':'Текущие больные'})
fig.add_trace(
    go.Scatter(
        name='14-дневная скользящая средняя',
        x=X[mask].Date,
        y=X[mask]['MA14_remaining_ill'],
        mode="lines",
        
        line=go.scatter.Line(color="green"),
        showlegend=True)
)

fig.add_trace(
    go.Scatter(
        name = '7-дневная скользящая средняя',
        x=X[mask].Date,
        y=X[mask]['MA7_remaining_ill'],
        mode="lines",
        
        line=go.scatter.Line(color="red"),
        showlegend=True)
)
fig.update_layout(
    annotations=[
#         dict(
#             x='2020-04-13',
#             y=10594,
#             xref="x",
#             yref="y",
#             text="Введение<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=0,
#             ay=-60),
#         dict(
#             x='2020-06-09',
#             y=85824,
#             xref="x",
#             yref="y",
#             text="Отмена<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=0,
#             ay=-60),
        
        dict(x=0.5,y=-0.15,
            showarrow=False,
            text="",
            xref="paper",
            yref="paper")],
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                  legend_x = 0.05,legend_y = 0.98,
                  width = 990, height = 600,template = 'gridon',
    
    
                  xaxis_title='Текущие больные = количество выявленных - количество выздоровевших - количество умерших'+
                                '<br>Данные: Яндекс',
    yaxis_title = ' '
)


fig.show()





In [12]:
mask = X.Region ==my_region
#mask = (X.Date>'2020-05-01')&(X.Region==my_region)
Y = X[mask]

#Y['base'] = Y['Change_dayconf']*Y['Change_dayconf'].apply(lambda d:0 if d>0 else 1)
Y['Date']= pd.to_datetime(Y.Date)
fig = go.Figure()
fig.add_trace(
              go.Bar(x = Y.Date,y=Y.Change_remill,text = Y['Change_remill'],textposition = 'inside',hoverinfo='x+y',
                
                #marker_color=Y['Change_remill'].apply(lambda d:'crimson' if d>0 else 'green'),
                marker_color = Y['Change_remill'],marker_colorscale = 'Temps',marker_colorbar ={'tickmode':'auto',
                                    'title':{'text':'Изменение','side':'top'}})     
                )
fig.update_layout(
    annotations=[
#         dict(
#             x='2020-04-13',
#             y=10594,
#             xref="x",
#             yref="y",
#             text="Введение<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=0,
#             ay=-60),
#         dict(
#             x='2020-06-09',
#             y=-3877,
#             xref="x",
#             yref="y",
#             text="Отмена<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=-20,
#             ay=40),
#         dict(
#             x='2020-05-20',
#             y=-2816,
#             xref="x",
#             yref="y",
#             text="20 мая",
#             showarrow=True,
#             arrowhead=2,
#             ax=-20,
#             ay=40),
        
        dict(x=0.5,y=-0.15,
            showarrow=False,
            text="",
            xref="paper",
            yref="paper")],
                  title=f'{my_region}: динамика текущих больных COVID-19 по дням<br>(изменение относительно предыдущего дня)',
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                  legend_x = 0.05,legend_y = 0.98,
                  width = 990, height = 600,template = 'gridon',
    
    
                  xaxis_title='Текущие больные = количество выявленных - количество выздоровевших - количество умерших'+
                               '<br>Данные : Яндекс',
    yaxis_title = ' '
)

fig.show()


In [13]:
delta = datetime.timedelta(14)
xday=myday-delta
print(xday)

2020-08-07


In [14]:
delta = datetime.timedelta(14)
past14=myday-delta
mask = (X.Date>=past14)&(X.Region.isin(regions_towatch))

def my_round(k):
    return round(k,3)

# #mask = (X.Date>='2020-06-04')

Z = X[mask].groupby(['Region','Date']).Rt.sum().apply(my_round).unstack().reset_index()
Z.dropna(inplace=True)

d = Z.drop('Region',axis = 1).values

# my_map= {'Москва':'2.Москва','Московская обл.':'3.Московская обл.','Нижегородская обл.':'5.Нижегородская обл.',
#                     'Россия':'1.Россия', 'Санкт-Петербург':'4.Санкт-Петербург'}
# Z.Region = Z.Region.map(my_map)

print(d)
Z.tail(10)
print(regions_towatch)

[[1.016 1.    1.024 1.003 1.018 1.037 1.    1.003 1.    0.995 0.987 0.966
  0.94  0.917 0.9  ]
 [0.931 0.905 0.901 0.911 0.917 0.949 0.956 0.951 0.94  0.938 0.952 0.968
  0.987 0.961 0.934]
 [1.002 1.004 1.005 1.002 1.007 1.007 1.007 1.001 0.999 0.999 0.997 1.001
  0.999 1.    1.   ]
 [1.039 1.021 0.98  0.967 0.964 0.95  0.968 0.982 0.987 1.005 1.015 0.992
  0.99  0.992 0.981]
 [0.959 0.976 0.987 0.987 0.98  0.973 0.967 0.972 0.991 0.99  0.988 0.975
  0.958 0.955 0.962]
 [0.98  0.977 0.991 0.994 0.998 1.013 1.006 1.013 1.021 1.014 1.013 1.006
  0.998 0.991 1.026]
 [0.969 0.987 0.997 1.024 1.024 1.021 1.013 0.997 1.005 1.008 1.013 1.026
  1.013 1.005 0.99 ]
 [0.9   0.927 0.956 0.972 0.977 0.967 0.932 0.903 0.904 0.894 0.892 0.919
  0.916 0.923 0.927]
 [0.973 1.099 1.105 1.105 1.139 0.958 0.901 0.895 0.914 1.072 1.188 1.17
  1.147 1.067 1.012]
 [0.991 0.949 0.995 0.946 1.005 1.005 0.943 0.953 0.931 0.951 1.01  1.057
  0.969 0.99  0.942]
 [0.878 0.861 0.854 0.867 0.897 0.943 0.939 0.955 0

In [15]:
x1=list(Z.columns[1:])
x2=[]
for i in x1:
    x2.append(str(i.day)+'.'+str(i.month)+"."+str(i.year))
    
y=list(Z.Region)
print(list(x1))
print('-'*40)
print(x2)

[datetime.date(2020, 8, 7), datetime.date(2020, 8, 8), datetime.date(2020, 8, 9), datetime.date(2020, 8, 10), datetime.date(2020, 8, 11), datetime.date(2020, 8, 12), datetime.date(2020, 8, 13), datetime.date(2020, 8, 14), datetime.date(2020, 8, 15), datetime.date(2020, 8, 16), datetime.date(2020, 8, 17), datetime.date(2020, 8, 18), datetime.date(2020, 8, 19), datetime.date(2020, 8, 20), datetime.date(2020, 8, 21)]
----------------------------------------
['7.8.2020', '8.8.2020', '9.8.2020', '10.8.2020', '11.8.2020', '12.8.2020', '13.8.2020', '14.8.2020', '15.8.2020', '16.8.2020', '17.8.2020', '18.8.2020', '19.8.2020', '20.8.2020', '21.8.2020']


In [16]:
fig = ff.create_annotated_heatmap(d, x=x2, y=y, annotation_text=d,hoverinfo='z',
                                  colorscale='Temps')

fig.update_layout(
                  
                  
                  width = 990, height = 600,    
    
                  xaxis_title='Динамика коэффициента распространения Rt по регионам',
               #   yaxis_title = 'Регион '
)

fig.show()



In [17]:
mask = (X.Date>=past14)&(X.Region.isin(regions_towatch))


# #mask = (X.Date>='2020-06-04')

Z = X[mask].groupby(['Region','Date']).Rt2.sum().apply(my_round).unstack().reset_index()
Z.dropna(inplace=True)
d = Z.drop('Region',axis = 1).values

# my_map= {'Москва':'2.Москва','Московская обл.':'3.Московская обл.','Нижегородская обл.':'5.Нижегородская обл.',
#                     'Россия':'1.Россия', 'Санкт-Петербург':'4.Санкт-Петербург'}
# Z.Region = Z.Region.map(my_map)

print(d)
Z.head(10)

[[1.041 1.04  1.04  1.04  1.04  1.04  1.039 1.039 1.038 1.038 1.038 1.037
  1.036 1.035 1.034]
 [1.04  1.039 1.037 1.036 1.035 1.034 1.034 1.033 1.032 1.031 1.031 1.03
  1.03  1.029 1.029]
 [1.011 1.011 1.011 1.011 1.011 1.011 1.011 1.011 1.011 1.011 1.011 1.011
  1.011 1.011 1.011]
 [1.01  1.01  1.01  1.01  1.01  1.01  1.009 1.009 1.009 1.009 1.009 1.009
  1.009 1.009 1.009]
 [1.025 1.025 1.024 1.024 1.024 1.024 1.023 1.023 1.023 1.023 1.022 1.022
  1.022 1.021 1.021]
 [1.02  1.02  1.02  1.02  1.019 1.019 1.019 1.019 1.019 1.019 1.019 1.019
  1.019 1.019 1.019]
 [1.039 1.039 1.038 1.038 1.038 1.038 1.038 1.037 1.037 1.037 1.036 1.036
  1.036 1.036 1.035]
 [1.036 1.035 1.034 1.034 1.033 1.033 1.032 1.031 1.03  1.029 1.028 1.027
  1.026 1.026 1.025]
 [1.052 1.053 1.054 1.054 1.055 1.054 1.052 1.05  1.048 1.048 1.05  1.051
  1.052 1.053 1.052]
 [1.046 1.045 1.044 1.043 1.043 1.042 1.041 1.04  1.039 1.038 1.038 1.038
  1.038 1.037 1.036]
 [1.036 1.035 1.033 1.032 1.03  1.03  1.029 1.029 1

Date,Region,2020-08-07,2020-08-08,2020-08-09,2020-08-10,2020-08-11,2020-08-12,2020-08-13,2020-08-14,2020-08-15,2020-08-16,2020-08-17,2020-08-18,2020-08-19,2020-08-20,2020-08-21
0,Волгоградская обл.,1.041,1.040,1.040,1.040,1.040,1.040,1.039,1.039,1.038,1.038,1.038,1.037,1.036,1.035,1.034
1,Иркутская обл.,1.040,1.039,1.037,1.036,1.035,1.034,1.034,1.033,1.032,1.031,1.031,1.030,1.030,1.029,1.029
2,Москва,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011
3,Московская обл.,1.010,1.010,1.010,1.010,1.010,1.010,1.009,1.009,1.009,1.009,1.009,1.009,1.009,1.009,1.009
4,Россия,1.025,1.025,1.024,1.024,1.024,1.024,1.023,1.023,1.023,1.023,1.022,1.022,1.022,1.021,1.021
5,Санкт-Петербург,1.020,1.020,1.020,1.020,1.019,1.019,1.019,1.019,1.019,1.019,1.019,1.019,1.019,1.019,1.019
6,Саратовская обл.,1.039,1.039,1.038,1.038,1.038,1.038,1.038,1.037,1.037,1.037,1.036,1.036,1.036,1.036,1.035
7,Свердловская обл.,1.036,1.035,1.034,1.034,1.033,1.033,1.032,1.031,1.030,1.029,1.028,1.027,1.026,1.026,1.025
8,Ставропольский край,1.052,1.053,1.054,1.054,1.055,1.054,1.052,1.050,1.048,1.048,1.050,1.051,1.052,1.053,1.052
9,Ульяновская обл.,1.046,1.045,1.044,1.043,1.043,1.042,1.041,1.040,1.039,1.038,1.038,1.038,1.038,1.037,1.036


In [18]:
x1=list(Z.columns[1:])
x2=[]
for i in x1:
    x2.append(str(i.day)+'.'+str(i.month)+"."+str(i.year))
    
y=list(Z.Region)
print(list(x1))
print(x2)

[datetime.date(2020, 8, 7), datetime.date(2020, 8, 8), datetime.date(2020, 8, 9), datetime.date(2020, 8, 10), datetime.date(2020, 8, 11), datetime.date(2020, 8, 12), datetime.date(2020, 8, 13), datetime.date(2020, 8, 14), datetime.date(2020, 8, 15), datetime.date(2020, 8, 16), datetime.date(2020, 8, 17), datetime.date(2020, 8, 18), datetime.date(2020, 8, 19), datetime.date(2020, 8, 20), datetime.date(2020, 8, 21)]
['7.8.2020', '8.8.2020', '9.8.2020', '10.8.2020', '11.8.2020', '12.8.2020', '13.8.2020', '14.8.2020', '15.8.2020', '16.8.2020', '17.8.2020', '18.8.2020', '19.8.2020', '20.8.2020', '21.8.2020']


In [19]:
fig = ff.create_annotated_heatmap(d, x=x2, y=y, annotation_text=d,
                                  colorscale='Temps')

fig.update_layout(
                  
                  
                  width = 990, height = 600,    
    
                  xaxis_title='Динамика коэффициента распространения Rt2 по регионам',
               #   yaxis_title = 'Регион '
)

fig.show()


In [20]:
mask = (X.Date>=past14)&(X.Region.isin(regions_towatch))

Z = X[mask].groupby(['Region','Date'])['Day_confirmed'].sum().unstack().reset_index()
Z.dropna(inplace=True)
d = Z.drop('Region',axis = 1).values
Z.head()

Date,Region,2020-08-07,2020-08-08,2020-08-09,2020-08-10,2020-08-11,2020-08-12,2020-08-13,2020-08-14,2020-08-15,2020-08-16,2020-08-17,2020-08-18,2020-08-19,2020-08-20,2020-08-21
0,Волгоградская обл.,93,88,106,97,95,91,102,97,96,92,95,89,87,84,82
1,Иркутская обл.,130,119,123,120,124,119,115,110,113,118,114,108,111,105,101
2,Москва,686,691,689,694,694,689,692,688,695,688,690,693,691,688,690
3,Московская обл.,158,161,149,153,154,152,148,156,153,154,153,145,151,157,151
4,Россия,5241,5212,5189,5118,4945,5102,5057,5065,5061,4969,4892,4748,4828,4785,4870


In [21]:



x1=list(Z.columns[1:])
y=list(Z.Region)
x2=[]
for i in x1:
    x2.append(str(i.day)+'.'+str(i.month)+"."+str(i.year))
    
print(x2)


['7.8.2020', '8.8.2020', '9.8.2020', '10.8.2020', '11.8.2020', '12.8.2020', '13.8.2020', '14.8.2020', '15.8.2020', '16.8.2020', '17.8.2020', '18.8.2020', '19.8.2020', '20.8.2020', '21.8.2020']


In [22]:
fig = ff.create_annotated_heatmap(d, x=x2, y=y, annotation_text=d,
                                  colorscale='Temps')

fig.update_layout(
                  
                  
                  width = 990, height = 600,
    
    
                  xaxis_title='Динамика новых заболевших по регионам',
               #   yaxis_title = 'Регион '
)

fig.show()

In [23]:
mask = (X.Date==myday)&(X.Region!='Россия')
#mask = (X.Date==myday)
#Делаем выборку по доле регина в общих случаях выявления заболеваний по России
df=X[mask].sort_values(by=['Rate_conf'],ascending = False)
#Далее заменяем все имена если регион набрал менее 1% в общих случаях по России
df2 = X[mask].reset_index(drop=True)
print(len(df2))

for i in range(len(df2)):
    if df2.loc[i,'Rate_conf']<1:
        df2.loc[i,'Region']='Регионы менее 1%'
df.head(15)

85


,Date,Confirmed,Recovered,Deaths,Day_deaths,Day_confirmed,Day_recovered,Rt,Rt2,Rate_conf,Rate_ill,Region,Remaining_ill,MA7_dayconfirmed,MA14_dayconfirmed,MA7_remaining_ill,MA14_remaining_ill,Change_dayconf,Change_remill
6857,2020-08-21,255826,202727,4710,12,690,1160,1.000,1.011,27.015046,28.555327,Москва,48389,690.714286,690.857143,49797.000000,51227.500000,2,-482
7030,2020-08-21,66692,50194,1160,3,151,134,0.981,1.009,7.042628,9.051264,Московская обл.,15338,152.000000,152.642857,15296.285714,15310.642857,-6,14
9817,2020-08-21,34828,24977,2314,6,181,101,1.026,1.019,3.677812,4.447736,Санкт-Петербург,7537,163.428571,160.928571,7332.571429,7084.928571,21,74
7520,2020-08-21,26391,23446,443,2,127,124,1.034,1.019,2.786871,1.476481,Нижегородская обл.,2502,125.857143,124.071429,2618.285714,2744.714286,-5,1
10469,2020-08-21,24105,19383,396,8,124,66,0.927,1.025,2.545471,2.552860,Свердловская обл.,4326,144.000000,157.142857,4137.714286,3879.285714,-15,50
12751,2020-08-21,18916,17157,155,1,85,168,0.869,1.018,1.997516,0.946553,ХМАО – Югра,1604,84.428571,101.714286,1873.857143,2118.642857,1,-84
3256,2020-08-21,15981,11838,239,0,101,116,0.934,1.029,1.687582,2.303829,Иркутская обл.,3904,110.000000,114.285714,3956.285714,3996.285714,-4,-15
5379,2020-08-21,15748,12808,410,7,117,76,0.979,1.031,1.662978,1.493004,Красноярский край,2530,116.571429,120.428571,2380.714286,2173.714286,3,34
9323,2020-08-21,15632,13159,301,0,128,156,0.998,1.034,1.650728,1.281741,Ростовская обл.,2172,127.571429,127.214286,2314.000000,2357.785714,1,-28
2278,2020-08-21,13150,11687,104,1,97,126,1.047,1.031,1.388631,0.801973,Воронежская обл.,1359,97.857143,96.428571,1449.000000,1609.357143,-2,-30


In [24]:



fig = px.pie(df2, values='Confirmed', names='Region',color_discrete_sequence=px.colors.sequential.Rainbow,
            title=f'Всего выявленных случаев заболевания COVID-19 в России: {df2.Confirmed.sum()}<br>'+
            f'(на дату: {myday})'
            ,
            width = 900, height=1000)

fig.update_traces(textinfo='percent+label')
fig.update_layout(
        
                #  title=f'{my_region}: динамика текущих больных COVID-19 по дням<br>(изменение относительно предыдущего дня)',
                  title_x = 0.5,
                  margin = {'t':150, 'b':0},
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                 # legend_x = 0.05,legend_y = 0.98,
                     
)

fig.show()


In [25]:
mask = (X.Date==myday)&(X.Region!='Россия')
#mask = (X.Date==myday)
#Делаем выборку по доле регина в общих случаях выявления заболеваний по России
df=X[mask].sort_values(by=['Rate_ill'],ascending = False)
#Далее заменяем все имена если регион набрал менее 1% в общих случаях по России
df2 = X[mask].reset_index(drop=True)
print(len(df2))

for i in range(len(df2)):
    if df2.loc[i,'Rate_ill']<1:
        df2.loc[i,'Region']='Регионы менее 1%'
df.head(20)

85


,Date,Confirmed,Recovered,Deaths,Day_deaths,Day_confirmed,Day_recovered,Rt,Rt2,Rate_conf,Rate_ill,Region,Remaining_ill,MA7_dayconfirmed,MA14_dayconfirmed,MA7_remaining_ill,MA14_remaining_ill,Change_dayconf,Change_remill
6857,2020-08-21,255826,202727,4710,12,690,1160,1.000,1.011,27.015046,28.555327,Москва,48389,690.714286,690.857143,49797.000000,51227.500000,2,-482
7030,2020-08-21,66692,50194,1160,3,151,134,0.981,1.009,7.042628,9.051264,Московская обл.,15338,152.000000,152.642857,15296.285714,15310.642857,-6,14
9817,2020-08-21,34828,24977,2314,6,181,101,1.026,1.019,3.677812,4.447736,Санкт-Петербург,7537,163.428571,160.928571,7332.571429,7084.928571,21,74
10469,2020-08-21,24105,19383,396,8,124,66,0.927,1.025,2.545471,2.552860,Свердловская обл.,4326,144.000000,157.142857,4137.714286,3879.285714,-15,50
13240,2020-08-21,13019,8682,102,1,73,65,0.928,1.023,1.374797,2.499159,Челябинская обл.,4235,72.285714,79.071429,4310.857143,4352.357143,2,7
9980,2020-08-21,11399,7305,68,1,91,109,0.990,1.035,1.203726,2.375824,Саратовская обл.,4026,96.000000,95.500000,4068.285714,3998.928571,-6,-19
3256,2020-08-21,15981,11838,239,0,101,116,0.934,1.029,1.687582,2.303829,Иркутская обл.,3904,110.000000,114.285714,3956.285714,3996.285714,-4,-15
1952,2020-08-21,11105,7462,83,1,82,119,0.900,1.034,1.172680,2.100828,Волгоградская обл.,3560,89.285714,92.928571,3695.142857,3658.214286,-2,-38
11121,2020-08-21,10741,7189,209,3,132,117,1.012,1.052,1.134242,1.972772,Ставропольский край,3343,130.285714,124.428571,3204.857143,3012.214286,3,12
12588,2020-08-21,11215,8293,80,4,91,238,0.942,1.036,1.184296,1.677122,Ульяновская обл.,2842,98.428571,98.571429,2983.857143,2923.642857,-8,-151


In [26]:


fig = px.pie(df2, values='Remaining_ill', names='Region',color_discrete_sequence=px.colors.sequential.Rainbow,
            title=f'Всего текущих больных COVID-19 в России: {df2.Remaining_ill.sum()}<br>'+
            f'(на дату: {myday})'
             ,
            width = 900, height=1000)

fig.update_traces(textinfo='percent+label')
fig.update_layout(
        
                #  title=f'{my_region}: динамика текущих больных COVID-19 по дням<br>(изменение относительно предыдущего дня)',
                  margin = {'t':150, 'b':0},
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                 # legend_x = 0.05,legend_y = 0.98,
                     
)

fig.show()


In [27]:
fig_r8 = go.Figure(data=[go.Pie(labels=df2.Region, values=df2.Remaining_ill, textinfo='label+percent',
                             insidetextorientation='radial'
                            )])
fig_r8.update_layout(
                title=f'Всего текущих больных COVID-19 в России: {df2.Remaining_ill.sum()}<br>'+
                f'(на дату: {myday})',
                 margin = {'t':120, 'b':0},
                  title_x = 0.5,
                  title_y= 0.9,
                  width = 900, height=900,
                  title_xanchor = "center",
                  title_yanchor = "bottom")
fig_r8.show()

In [28]:
delta2 = datetime.timedelta(7)
past7=myday-delta2
mask = (X.Date>=past7)&(X.Region.isin(regions_towatch))&(X.Region!='Россия')
                                      
box_cases1 = X[mask].groupby(['Region','Date'])['Day_confirmed'].sum()
box_cases2 = X[mask].groupby(['Region','Date'])['Day_recovered'].sum()
for i in regions_dict.keys():
    if regions_dict.get(i) in regions_towatch:
        print(i,regions_dict.get(i))

12 Волгоградская обл.
20 Иркутская обл.
42 Москва
43 Московская обл.
56 Россия
60 Санкт-Петербург
61 Саратовская обл.
64 Свердловская обл.
68 Ставропольский край
77 Ульяновская обл.
81 Челябинская обл.


In [29]:
new_df1=box_cases1.unstack().T
new_df1.tail(20)

Region,Волгоградская обл.,Иркутская обл.,Москва,Московская обл.,Санкт-Петербург,Саратовская обл.,Свердловская обл.,Ставропольский край,Ульяновская обл.,Челябинская обл.
Date,,,,,,,,,,
2020-08-14,97,110,688,156,159,94,154,130,86,80
2020-08-15,96,113,695,153,162,97,158,127,120,76
2020-08-16,92,118,688,154,163,98,150,129,91,79
2020-08-17,95,114,690,153,159,96,141,132,102,69
2020-08-18,89,108,693,145,157,98,149,128,95,67
2020-08-19,87,111,691,151,162,95,147,135,91,71
2020-08-20,84,105,688,157,160,97,139,129,99,71
2020-08-21,82,101,690,151,181,91,124,132,91,73


In [30]:

cols = list(new_df1.columns)
color_dict=dict(zip(cols,['forestgreen','darkblue','goldenrod','magenta','hotpink','grey','maroon','coral','darkorange','lightpink']))
#cols = cols[:-1]

fig = go.Figure()
for i in cols:
    fig.add_trace(
    go.Box(x=new_df1[i],name = i,boxmean = True,marker_color = color_dict.get(i)))

fig.update_layout(title_text="Статистика по новым заболевшим за последнюю неделю",
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "top",
                  xaxis_title='Новые заболевшие', yaxis_title='Регионы',width = 990, height = 600)
fig.show()

In [31]:
new_df2=box_cases2.unstack().T
new_df2.tail(20)

Region,Волгоградская обл.,Иркутская обл.,Москва,Московская обл.,Санкт-Петербург,Саратовская обл.,Свердловская обл.,Ставропольский край,Ульяновская обл.,Челябинская обл.
Date,,,,,,,,,,
2020-08-14,68,146,1350,159,72,81,75,59,126,105
2020-08-15,97,109,1275,156,76,60,77,69,89,98
2020-08-16,16,16,1045,138,53,19,76,22,23,104
2020-08-17,12,37,819,148,69,17,71,28,15,3
2020-08-18,135,226,1155,118,83,158,74,64,97,127
2020-08-19,154,119,1242,135,86,134,73,90,73,112
2020-08-20,192,126,1208,140,83,113,68,79,175,137
2020-08-21,119,116,1160,134,101,109,66,117,238,65


In [32]:

cols = list(new_df2.columns)
#cols = cols[:-1]

fig = go.Figure()
for i in cols:
    fig.add_trace(
    go.Box(x=new_df2[i],name = i,boxmean=True,marker_color = color_dict.get(i)))

fig.update_layout(title_text="Статистика по выздоровевшим за последнюю неделю",
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "top",
                  xaxis_title='Выздоровевшие', yaxis_title='Регионы',width = 990, height = 600)
fig.show()

In [33]:
from plotly.subplots import make_subplots

# fig = tools.make_subplots(rows = 1, cols = 2, specs = [[{}, {}]],
#                           horizontal_spacing = 0.05)


fig = make_subplots(rows=2, cols=1,specs = [[{}],[{}]],vertical_spacing = 0.04,shared_xaxes=True,
                  subplot_titles=("Box-график распределения по новым заболевшим(в день) за неделю:",
                                 "Box-график распределения по выздоровевшим(в день) за неделю:"))
#subplot_titles=("Статистика по новым заболевшим за последние 2 недели","Статистика по выздоровевшим за последние 2 недели")

for i in cols:
    fig.add_trace(
    go.Box(x=new_df1[i],name = i,boxmean = True,marker_color = color_dict.get(i)),row=1,col=1)

for i in cols:
    fig.add_trace(
    go.Box(x=new_df2[i],name = i,boxmean=True,marker_color = color_dict.get(i)),row=2,col=1)


fig.update_layout(
#     annotations=[
#         dict(
#             x=0.5,
#             y=-0.15,
#             showarrow=False,
#             text="Ветикальная пунктирная линия - среднее, Вертикальная сплошная - медиана распределения",
#             xref="paper",
#             yref="paper"),
        
#                 ],
    
    
                  width = 990, height = 1200,
    
    
                 showlegend=False,template ='ggplot2'
)

#fig.update_layout(height=1000, width=800,showlegend=False)
fig.show()